Convolutional Layer

In [13]:
import numpy as np

# Convolutional Layer
def conv2d(X, filters, stride=1, padding=0):
    n_filters, filter_depth, fh, fw = filters.shape
    X_depth, X_h, X_w = X.shape

    # Calculate output dimensions
    out_h = (X_h - fh + 2 * padding) // stride + 1
    out_w = (X_w - fw + 2 * padding) // stride + 1

    # Initialize output
    output = np.zeros((n_filters, out_h, out_w))

    # Pad the input if needed
    if padding > 0:
        X_padded = np.pad(X, ((0, 0), (padding, padding), (padding, padding)), mode='constant')
    else:
        X_padded = X

    # Perform convolution
    for f in range(n_filters):
        for i in range(out_h):
            for j in range(out_w):
                h_start = i * stride
                h_end = h_start + fh
                w_start = j * stride
                w_end = w_start + fw
                output[f, i, j] = np.sum(X_padded[:, h_start:h_end, w_start:w_end] * filters[f])
    return output


Max Pooling

In [14]:
# Max Pooling Layer
def max_pool2d(X, pool_size=(2, 2), stride=2):
    X_depth, X_h, X_w = X.shape
    pool_h, pool_w = pool_size

    out_h = (X_h - pool_h) // stride + 1
    out_w = (X_w - pool_w) // stride + 1

    output = np.zeros((X_depth, out_h, out_w))

    for d in range(X_depth):
        for i in range(out_h):
            for j in range(out_w):
                h_start = i * stride
                h_end = h_start + pool_h
                w_start = j * stride
                w_end = w_start + pool_w
                output[d, i, j] = np.max(X[d, h_start:h_end, w_start:w_end])

    return output

# Flatten Layer: reshape into (batch_size, features)
def flatten(X):
    return X.reshape(1, -1)  # for a single sample, returns shape (1, channels*height*width)

# Dense Layer
def dense(X, weights, bias, activation=None):
    output = np.dot(X, weights) + bias
    if activation == 'relu':
        output = relu(output)
    elif activation == 'softmax':
        output = softmax(output)
    return output

# ReLU Activation
def relu(X):
    return np.maximum(0, X)

# Softmax Activation
def softmax(X):
    exp_X = np.exp(X - np.max(X))
    return exp_X / np.sum(exp_X)

Flatten/Dense Layer, ReLU Activation, Softmax funtion

In [15]:
class CNN:
    def __init__(self):
        # First convolution: input channels = 1, output channels = 32
        self.filters1 = np.random.randn(32, 1, 3, 3) * 0.01  
        # Second convolution: input channels = 32, output channels = 64
        self.filters2 = np.random.randn(64, 32, 3, 3) * 0.01  
        # Third convolution: input channels = 64, output channels = 64
        self.filters3 = np.random.randn(64, 64, 3, 3) * 0.01  
        
        # For a 28x28 input:
        # After first conv (padding=1) and pool → shape: (32, 14, 14)
        # After second conv (padding=1) and pool → shape: (64, 7, 7)
        # After third conv (padding=1) → shape: (64, 7, 7)
        flattened_size = 64 * 7 * 7
        
        # Dense layers
        self.dense_weights1 = np.random.randn(flattened_size, 64) * 0.01
        self.dense_bias1 = np.zeros(64)
        
        self.dense_weights2 = np.random.randn(64, 10) * 0.01
        self.dense_bias2 = np.zeros(10)
    
    def forward(self, X):
        # X is expected to have shape (channels, height, width)
        # First Convolutional Block
        X = conv2d(X, self.filters1, stride=1, padding=1)  # (32, 28, 28)
        X = relu(X)
        X = max_pool2d(X, pool_size=(2, 2), stride=2)        # (32, 14, 14)
        
        # Second Convolutional Block
        X = conv2d(X, self.filters2, stride=1, padding=1)    # (64, 14, 14)
        X = relu(X)
        X = max_pool2d(X, pool_size=(2, 2), stride=2)          # (64, 7, 7)
        
        # Third Convolutional Block
        X = conv2d(X, self.filters3, stride=1, padding=1)    # (64, 7, 7)
        X = relu(X)
        
        # Flatten the output
        X_flat = flatten(X)  # Now shape is (1, 3136)
        
        # Dense Layers
        X_dense = dense(X_flat, self.dense_weights1, self.dense_bias1, activation='relu')
        output = dense(X_dense, self.dense_weights2, self.dense_bias2, activation='softmax')
        
        return output

CNN Model

In [16]:
# CNN Model
class CNN:
    def __init__(self):
        # First convolution: input channels = 1, output channels = 32
        self.filters1 = np.random.randn(32, 1, 3, 3) * 0.01  
        # Second convolution: input channels = 32, output channels = 64
        self.filters2 = np.random.randn(64, 32, 3, 3) * 0.01  
        # Third convolution: input channels = 64, output channels = 64
        self.filters3 = np.random.randn(64, 64, 3, 3) * 0.01  
        
        # For a 28x28 input:
        # After first conv (padding=1) and pool → size remains 28 -> pool reduces to 14x14
        # After second conv (padding=1) and pool → 14 -> pool reduces to 7x7
        # After third conv (padding=1) → remains 7x7 with 64 channels
        flattened_size = 64 * 7 * 7
        
        # Dense layers
        self.dense_weights1 = np.random.randn(flattened_size, 64) * 0.01
        self.dense_bias1 = np.zeros(64)
        
        self.dense_weights2 = np.random.randn(64, 10) * 0.01
        self.dense_bias2 = np.zeros(10)
    
    def forward(self, X):
        # X is expected to have shape (channels, height, width)
        # First Convolutional Block
        X = conv2d(X, self.filters1, stride=1, padding=1)  # Output shape: (32, 28, 28)
        X = relu(X)
        X = max_pool2d(X, pool_size=(2, 2), stride=2)        # Output shape: (32, 14, 14)
        
        # Second Convolutional Block
        X = conv2d(X, self.filters2, stride=1, padding=1)    # Output shape: (64, 14, 14)
        X = relu(X)
        X = max_pool2d(X, pool_size=(2, 2), stride=2)          # Output shape: (64, 7, 7)
        
        # Third Convolutional Block
        X = conv2d(X, self.filters3, stride=1, padding=1)    # Output shape: (64, 7, 7)
        X = relu(X)
        
        # Flatten
        X_flat = flatten(X)  # shape: (64*7*7,)
        
        # Dense Layers
        X_dense = dense(X_flat, self.dense_weights1, self.dense_bias1, activation='relu')
        output = dense(X_dense, self.dense_weights2, self.dense_bias2, activation='softmax')
        
        return output

Testing

In [17]:
# Example input: a 28x28 image with 1 channel (grayscale)
X = np.random.randn(1, 28, 28)  # shape: (channels, height, width)

# Initialize the CNN model and perform a forward pass
model = CNN()
output = model.forward(X)

print("Output shape:", output.shape)
print("Output probabilities:", output)

Output shape: (1, 10)
Output probabilities: [[0.09999855 0.1000011  0.09999965 0.10000329 0.09999672 0.10000317
  0.09999878 0.09999901 0.10000066 0.09999907]]
